Каждый из файлов субтитров в датасете OpenSubtitles [2], который мы использовали в качестве источника реплик и разговоров, содержит упорядоченный набор реплик. В большинстве случаев, каждая реплика – это ответ на предыдущую, в разговоре между двумя персонажами фильма. Мы случайно выбрали эпизоды этих разговоров в качестве наших тренировочных и тестовых примеров.

Каждый эпизод состоит из двух частей – контекста (Context) и финальной реплики (Reply). Например,

**context_2:** Персонаж A говорит реплику   
**context_1:** Персонаж B отвечает на нее   
**context_0:** Персонаж А произносит вторую реплику  
**reply:** Персонаж B отвечает на вторую реплику  
Контекстная часть может состоять из трех реплик (как в примере) – в 50% случаев, двух – в 25%, и одного – в оставшихся 25% случаев. Финальная реплика (Reply) всегда завершает любой эпизод, то есть следует за контекстом (Context). Задача участников – найти наиболее подходящую и интересную реплику для данного контекста среди предложенных кандидатов (числом до 6), случайно выбранных из топа кандидатов, возвращенных бейзлайном высокого качества, натренированным командой Алисы (который, в свою очередь, отобрал кандидатов среди всех возможных реплик OpenSubtitles).  

Все реплики-кандидаты размечены асессорами на сервисе Яндекс.Толока с помощью следующей инструкции для разметки:  

- **Good (2):** реплика уместна (имеет смысл для данного контекста) и интересна (нетривиальна, специфична именно для данного контекста, мотивирует продолжать разговор)  
- **Neutral (1):** реплика уместна (имеет смысл для данного контекста), но не интересна (тривиальна, не специфична для данного контекста и скорее подталкивает пользователя закончить разговор)  
- **Bad (0):** реплика не имеет никакого смысла в данном контексте  
Каждая метка в тренировочной части датасета (и только в ней), сопровождается также уверенностью (confidence) – числом в интервале от 0 до 1 – которое показывает насколько уверенными в своей разметке были асессоры с Толоки, совместно предложившие данную метку. Мы хотим обратить особое внимание участников на эту информацию, она может быть очень полезна при обучении их моделей.  

Мы хотим особо отметить, что все участники имеют право скачать датасет OpenSubtitles [2], который использовался для подготовки датасета и применять его для тренировки своих моделей по своему усмотрению.

Чтобы скачать данные, перейдите по ссылке /algorithm2018/contest/7914/download/1/ или Скачать условие задачи под описанием задачи.

Каждая строка в тренировочной части датасета представлена в следующем формате:

- **context_id** – идентификатор эпизода  
- **context_2,context_1,context_0** – текст реплик, предшествующих финальной (может состоять из трех частей)  
- **reply_id** – идентификатор реплики-кандидата  
- **reply** – текст реплики-кандидата  
- **label** – метка реплики-кандидата (good, neutral или bad)  
- **confidence** - уверенность в метке реплики-кандидата (число от 0 до 1)  
Каждая строка в тестовой части датасета представлена в следующем формате (по аналогии с тренировочной, но без информации о метках):  

- **context_id,context_2,context_1,context_0,reply_id,reply**  

Все строки в файле, который присылают участники должны быть организованы следующим образом:

- **context_id, reply_id**

где все context_id должны быть отсортированы на уровне посылаемого файла в возрастающем порядке
и все reply_id должны быть в порядке ранжирования реплик (то есть, в порядке убывания их скоров), который возвратила ваша система для данного context_id
context_id и reply_id должны быть отделены либо символом пробела либо tab
каждый файл-решение должен содержать то же число строк, что и файл с тестовыми данными
Обратите внимание, что финальный тестсет, для которого участники должны будут прислать свое ранжирование будет выложен за 48 часов до окончания конкурса.



# Метрика
Задача участников – возвратить ранжирование реплик-кандидатов представленных в порядке убывания скоров, выданных моделями участников. Метрика для оценивания этих ранжирований – NDCG. Больше информации о метрике можно найти на вики-странице https://en.wikipedia.org/wiki/Discounted_cumulative_gain - обратите внимание, что мы используем первый из двух вариантов DCG, представленных на странице.
![image.png](https://contest.yandex.ru/testsys/statement-image?imageId=a3d60fc7162405b413487b0b1bb2d46e438be4a018a50c1768f4b8d576750cda)

**IDCG** – это максимально возможное значение метрики DCG для данного набора кандидатов, оно измеряется после ранжирования кандидатов в порядке убывания значений их меток (не предсказанных скоров).

**reli** принимает три возможных значения - 2, 1 и 0 - для меток **good, neutral** и **bad** соответственно.

Особо отмечаем, что информация об уверенности меток (доступная только для тренировочных данных) никак не учитывается в метрике.

Скор участников отображаемый в контесте - это среднее **NDCG** для всех **context_id** тестовых данных, умноженное на 100 000.



In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 1500

# Read

In [3]:
train = pd.read_csv(
    './data/train.tsv',
    names=[
        'context_id', 'context_2', 'context_1', 'context_0', 'reply_id',
        'reply', 'label', 'confidence'
    ],
    sep='\t', quoting=3, header=None, encoding="utf-8")

In [4]:
test = pd.read_csv(
    './data/public.tsv',
    names=[
        'context_id', 'context_2', 'context_1', 'context_0', 'reply_id',
        'reply'
    ],
    sep='\t', quoting=3, error_bad_lines=False, header=None, encoding="utf-8")

In [5]:
train.head(3)

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320


In [6]:
test.head(3)

,context_id,context_2,context_1,context_0,reply_id,reply
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .


In [7]:
train.shape

(97533, 8)

# Train

In [73]:
import fastText
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [43]:
import pymorphy2 as morphy
from pymorphy2.tokenizers import simple_word_tokenize

In [9]:
from sklearn.metrics import mean_absolute_error

In [10]:
ft_model = fastText.load_model("./data/wiki.ru/wiki.ru.bin")

### Features

In [11]:
def rank2num(st):
    return {'good': 2, 'neutral': 1, 'bad': 0}[st]

In [12]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [13]:
train['rank'] = train['label'].apply(rank2num)

In [14]:
train['target'] = train['rank'] * train['confidence']

In [157]:
train['summed_str'] = train['context_2'] + train['context_1'] + train['context_0']
test['summed_str'] = test['context_2'] + test['context_1'] + test['context_0']

#### Language

In [489]:
def count_symb(sentence):
    return sum(1 for x in sentence if x in '.,<>?!@#$%^&*()/\;:')

def count_vow(sentence):
    return sum(1 for x in sentence.lower() if x in 'аеёиоюэя')

def count_consn(sentence):
    return sum(1 for x in sentence.lower() if x in 'бвгджзклмнпрстфхцчшщ')

def count_ship(sentence):
    return sum(1 for x in sentence.lower() if x in 'жчшщ')

def count_swist(sentence):
    return sum(1 for x in sentence.lower() if x in 'цзс')

def count_zvon_parn(sentence):
    return sum(1 for x in sentence.lower() if x in 'бвгджз')

def count_glukh_parn(sentence):
    return sum(1 for x in sentence.lower() if x in 'пфктшс')

def count_zvon_neparn(sentence):
    return sum(1 for x in sentence.lower() if x in 'лмнрй')

def count_glukh_neparn(sentence):
    return sum(1 for x in sentence.lower() if x in 'хцчщ')

def count_low(sentence):
    return sum(1 for x in sentence if x.islower())

def count_up(sentence):
    return sum(1 for x in sentence if x.isupper())

def count_words(sentence):
    return len(sentence.split(' '))

funcs_dict = {
    'count_symb': count_symb,
    'count_vow': count_vow,
    'count_consn': count_consn,
    'count_ship': count_ship,
    'count_swist': count_swist,
    'count_zvon_parn': count_zvon_parn,
    'count_glukh_parn': count_glukh_parn,
    'count_zvon_neparn': count_zvon_neparn,
    'count_glukh_neparn': count_glukh_neparn,
    'count_low': count_low,
    'count_up': count_up,
    'count_words':count_words,
}

In [158]:
for key, c_function in funcs_dict.items():
    train[key] = train['summed_str'].apply(c_function)
for key, c_function in funcs_dict.items():
    train['reply_' + key] = train['reply'].apply(c_function)

In [159]:
for key, c_function in funcs_dict.items():
    test[key] = test['summed_str'].apply(c_function)
for key, c_function in funcs_dict.items():
    test['reply_' + key] = test['reply'].apply(c_function)

#### Counts

In [464]:
train['cont_2_words'] = scale(train['context_2'].apply(count_words))
train['cont_1_words'] = scale(train['context_1'].apply(count_words))
train['cont_0_words'] = scale(train['context_0'].apply(count_words))
train['reply_words'] = scale(train['reply'].apply(count_words))

test['cont_2_words'] = scale(test['context_2'].apply(count_words))
test['cont_1_words'] = scale(test['context_1'].apply(count_words))
test['cont_0_words'] = scale(test['context_0'].apply(count_words))
test['reply_words'] = scale(test['reply'].apply(count_words))

/Users/solo/.pyenv/versions/jupyter3.6.4/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [287]:
train['context_2_len'] = train['context_2'].apply(lambda x: 1 if len(x) > 0 else 0)
train['context_1_len'] = train['context_1'].apply(lambda x: 1 if len(x) > 0 else 0)
train['context_0_len'] = train['context_0'].apply(lambda x: 1 if len(x) > 0 else 0)
train['context_count'] = train['context_2_len'] + train['context_1_len'] + train['context_0_len']

In [288]:
test['context_2_len'] = test['context_2'].apply(lambda x: 1 if len(x) > 0 else 0)
test['context_1_len'] = test['context_1'].apply(lambda x: 1 if len(x) > 0 else 0)
test['context_0_len'] = test['context_0'].apply(lambda x: 1 if len(x) > 0 else 0)
test['context_count'] = test['context_2_len'] + test['context_1_len'] + test['context_0_len']

#### Morph

In [44]:
analyzer = morphy.MorphAnalyzer()

In [45]:
def count_poses(string):
    interesting = {
        'NOUN':0,
        'ADJ':0,
        'VERB':0,
        'ADVB': 0,
        'PRTF': 0,
        'NPRO': 0,
        'NUMR': 0,
        'GRND': 0,
        'NUMB': 0,
        'LATN': 0
    }
    for token in simple_word_tokenize(string):
        pos = analyzer.tag(token)[0].POS
        if pos == 'ADJF' or pos == 'ADJS':
            interesting['ADJ'] +=1
            continue
        if pos == 'VERB' or pos == 'INFN':
            interesting['VERB'] +=1
            continue
        if pos == 'PRTF' or 'PRTS':
            interesting['PRTF'] += 1
            continue
        if pos in interesting:
            interesting[pos] += 1
    return interesting

In [289]:
test.columns

Index(['context_id', 'context_2', 'context_1', 'context_0', 'reply_id',
       'reply', 'cont_2_words', 'cont_1_words', 'cont_0_words', 'reply_words',
       'context_2_len', 'context_1_len', 'context_0_len', 'context_count'],
      dtype='object')

In [15]:
train.head(2)

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,2,1.750703
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,1,0.900968


### vstack

#### Additional exp

In [163]:
t6_ft = np.vstack(train['count_symb'])
t7_ft = np.vstack(train['count_vow'])
t8_ft = np.vstack(train['count_consn'])
t9_ft = np.vstack(train['count_ship'])
t10_ft = np.vstack(train['count_swist'])
t11_ft = np.vstack(train['count_zvon_parn'])
t12_ft = np.vstack(train['count_glukh_parn'])
t13_ft = np.vstack(train['count_zvon_neparn'])
t14_ft = np.vstack(train['count_glukh_neparn'])
t15_ft = np.vstack(train['count_low'])
t16_ft = np.vstack(train['count_up'])
t16_ft = np.vstack(train['reply_count_symb'])
t17_ft = np.vstack(train['reply_count_vow'])
t18_ft = np.vstack(train['reply_count_consn'])
t19_ft = np.vstack(train['reply_count_ship'])
t20_ft = np.vstack(train['reply_count_swist'])
t21_ft = np.vstack(train['reply_count_zvon_parn'])
t22_ft = np.vstack(train['reply_count_glukh_parn'])
t23_ft = np.vstack(train['reply_count_zvon_neparn'])
t24_ft = np.vstack(train['reply_count_glukh_neparn'])
t25_ft = np.vstack(train['reply_count_low'])
t26_ft = np.vstack(train['reply_count_up'])
t27_ft = np.vstack(train['count_words'])
t28_ft = np.vstack(train['reply_count_words'])

te6_ft = np.vstack(test['count_symb'])
te7_ft = np.vstack(test['count_vow'])
te8_ft = np.vstack(test['count_consn'])
te9_ft = np.vstack(test['count_ship'])
te10_ft = np.vstack(test['count_swist'])
te11_ft = np.vstack(test['count_zvon_parn'])
te12_ft = np.vstack(test['count_glukh_parn'])
te13_ft = np.vstack(test['count_zvon_neparn'])
te14_ft = np.vstack(test['count_glukh_neparn'])
te15_ft = np.vstack(test['count_low'])
te16_ft = np.vstack(test['count_up'])
te16_ft = np.vstack(test['reply_count_symb'])
te17_ft = np.vstack(test['reply_count_vow'])
te18_ft = np.vstack(test['reply_count_consn'])
te19_ft = np.vstack(test['reply_count_ship'])
te20_ft = np.vstack(test['reply_count_swist'])
te21_ft = np.vstack(test['reply_count_zvon_parn'])
te22_ft = np.vstack(test['reply_count_glukh_parn'])
te23_ft = np.vstack(test['reply_count_zvon_neparn'])
te24_ft = np.vstack(test['reply_count_glukh_neparn'])
te25_ft = np.vstack(test['reply_count_low'])
te26_ft = np.vstack(test['reply_count_up'])
te27_ft = np.vstack(test['count_words'])
te28_ft = np.vstack(test['reply_count_words'])

In [465]:
t29_ft = np.vstack(train['cont_2_words'])
t30_ft = np.vstack(train['cont_1_words'])
t31_ft = np.vstack(train['cont_0_words'])
t32_ft = np.vstack(train['reply_words'])

te29_ft = np.vstack(test['cont_2_words'])
te30_ft = np.vstack(test['cont_1_words'])
te31_ft = np.vstack(test['cont_0_words'])
te32_ft = np.vstack(test['reply_words'])

In [398]:
t33_ft = np.vstack(train['context_count'])

te33_ft = np.vstack(test['context_count'])

#### Morph

In [46]:
train['pymorphy_reply'] = train['reply'].apply(lambda x: count_poses(x))
t34_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['NOUN']))
t35_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['ADJ']))
t36_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['VERB']))
t37_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['ADVB']))
t38_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['PRTF']))
t39_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['NPRO']))
t40_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['NUMR']))
t41_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['GRND']))
t42_ft = np.vstack(train['pymorphy_reply'].apply(lambda x: x['NUMB']))
train.drop('pymorphy_reply' , axis=1 , inplace=True)
train['pymorphy_context_0'] = train['context_0'].apply(lambda x: count_poses(x))
t43_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['NOUN']))
t44_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['ADJ']))
t45_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['VERB']))
t46_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['ADVB']))
t47_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['PRTF']))
t48_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['NPRO']))
t49_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['NUMR']))
t50_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['GRND']))
t51_ft = np.vstack(train['pymorphy_context_0'].apply(lambda x: x['NUMB']))
train.drop('pymorphy_context_0' , axis=1 , inplace=True)

test['pymorphy_reply'] = test['reply'].apply(lambda x: count_poses(x))
te34_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['NOUN']))
te35_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['ADJ']))
te36_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['VERB']))
te37_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['ADVB']))
te38_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['PRTF']))
te39_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['NPRO']))
te40_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['NUMR']))
te41_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['GRND']))
te42_ft = np.vstack(test['pymorphy_reply'].apply(lambda x: x['NUMB']))
test.drop('pymorphy_reply' , axis=1 , inplace=True)
test['pymorphy_context_0'] = test['context_0'].apply(lambda x: count_poses(x))
te43_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['NOUN']))
te44_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['ADJ']))
te45_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['VERB']))
te46_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['ADVB']))
te47_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['PRTF']))
te48_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['NPRO']))
te49_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['NUMR']))
te50_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['GRND']))
te51_ft = np.vstack(test['pymorphy_context_0'].apply(lambda x: x['NUMB']))
test.drop('pymorphy_context_0' , axis=1 , inplace=True)

In [54]:
train['pymorphy_context_1'] = train['context_1'].apply(lambda x: count_poses(x))
t52_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['NOUN']))
t53_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['ADJ']))
t54_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['VERB']))
t55_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['ADVB']))
t56_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['PRTF']))
t57_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['NPRO']))
t58_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['NUMR']))
t59_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['GRND']))
t60_ft = np.vstack(train['pymorphy_context_1'].apply(lambda x: x['NUMB']))
train.drop('pymorphy_context_1' , axis=1 , inplace=True)
print('train pymorphy_context_1 done')
test['pymorphy_context_1'] = test['context_1'].apply(lambda x: count_poses(x))
te52_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['NOUN']))
te53_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['ADJ']))
te54_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['VERB']))
te55_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['ADVB']))
te56_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['PRTF']))
te57_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['NPRO']))
te58_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['NUMR']))
te59_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['GRND']))
te60_ft = np.vstack(test['pymorphy_context_1'].apply(lambda x: x['NUMB']))
test.drop('pymorphy_context_1' , axis=1 , inplace=True)
print('test pymorphy_context_1 done')
train['pymorphy_context_2'] = train['context_2'].apply(lambda x: count_poses(x))
t61_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['NOUN']))
t62_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['ADJ']))
t63_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['VERB']))
t64_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['ADVB']))
t65_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['PRTF']))
t66_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['NPRO']))
t67_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['NUMR']))
t68_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['GRND']))
t69_ft = np.vstack(train['pymorphy_context_2'].apply(lambda x: x['NUMB']))
train.drop('pymorphy_context_2' , axis=1 , inplace=True)
print('train pymorphy_context_2 done')
test['pymorphy_context_2'] = test['context_2'].apply(lambda x: count_poses(x))
te61_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['NOUN']))
te62_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['ADJ']))
te63_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['VERB']))
te64_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['ADVB']))
te65_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['PRTF']))
te66_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['NPRO']))
te67_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['NUMR']))
te68_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['GRND']))
te69_ft = np.vstack(test['pymorphy_context_2'].apply(lambda x: x['NUMB']))
test.drop('pymorphy_context_2' , axis=1 , inplace=True)
print('test pymorphy_context_2 done')

train pymorphy_context_1 done
test pymorphy_context_1 done
train pymorphy_context_2 done
test pymorphy_context_2 done


#### Baseline

In [16]:
t1_ft = np.vstack(train['context_2'].apply(lambda x: ft_model.get_sentence_vector(x)))
t2_ft = np.vstack(train['context_1'].apply(lambda x: ft_model.get_sentence_vector(x)))
t3_ft = np.vstack(train['context_0'].apply(lambda x: ft_model.get_sentence_vector(x)))
t5_ft = np.vstack(train['reply'].apply(lambda x: ft_model.get_sentence_vector(x)))

te1_ft = np.vstack(test['context_2'].apply(lambda x: ft_model.get_sentence_vector(x)))
te2_ft = np.vstack(test['context_1'].apply(lambda x: ft_model.get_sentence_vector(x)))
te3_ft = np.vstack(test['context_0'].apply(lambda x: ft_model.get_sentence_vector(x)))
te5_ft = np.vstack(test['reply'].apply(lambda x: ft_model.get_sentence_vector(x)))

In [92]:
X_train = np.hstack([
    t1_ft, t2_ft, t3_ft, t5_ft, 
#     t6_ft, t7_ft, t8_ft, t9_ft, t10_ft, t11_ft,
#     t12_ft, t13_ft, t14_ft, t15_ft, t16_ft, 
#     t16_ft, t17_ft, 
#     t18_ft, t19_ft, t20_ft, t21_ft, t22_ft, t23_ft, t24_ft, t25_ft, t26_ft, 
#     t27_ft, t28_ft,
#     t29_ft, t30_ft, t31_ft, t32_ft,
#     t33_ft,
    t34_ft, t35_ft, t36_ft, t37_ft, t38_ft, t39_ft, t40_ft, t41_ft, t42_ft, 
    t43_ft, t44_ft, t45_ft, t46_ft, t47_ft, t48_ft, t49_ft, t50_ft, t51_ft, 
    t52_ft, t53_ft, t54_ft, t55_ft, t56_ft, t57_ft, t58_ft, t59_ft, t60_ft,
    t61_ft, t62_ft, t63_ft, t64_ft, t65_ft, t66_ft, t67_ft, t68_ft, t69_ft,
])

X_test = np.hstack([
    te1_ft, te2_ft, te3_ft, te5_ft, 
#     te6_ft, te7_ft, te8_ft, te9_ft, te10_ft,
#     te11_ft, te12_ft, te13_ft, te14_ft, te15_ft, 
#     te16_ft, te17_ft,
#     te18_ft, te19_ft, te20_ft, te21_ft, te22_ft, te23_ft, te24_ft, te25_ft, te26_ft, 
#     te27_ft, te28_ft
#     te29_ft, te30_ft, te31_ft, te32_ft,
#     te33_ft,
    te34_ft, te35_ft, te36_ft, te37_ft, te38_ft, te39_ft, te40_ft, te41_ft, te42_ft, 
    te43_ft, te44_ft, te45_ft, te46_ft, te47_ft, te48_ft, te49_ft, te50_ft, te51_ft,
    te52_ft, te53_ft, te54_ft, te55_ft, te56_ft, te57_ft, te58_ft, te59_ft, te60_ft,
    te61_ft, te62_ft, te63_ft, te64_ft, te65_ft, te66_ft, te67_ft, te68_ft, te69_ft, 
])

In [104]:
X_train.shape

(97533, 1236)

In [57]:
y_train = train['target']

In [58]:
train_part_size = int(0.75 * train['target'].shape[0])
X_train_part = X_train[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid =  X_train[train_part_size:, :]
y_valid = y_train[train_part_size:]

## Stack models

In [59]:
def form_last_vec(models, X_vec):
    return np.array([
        models['m_00'].predict(X_vec),
        models['m_01'].predict(X_vec),
        models['m_02'].predict(X_vec),
    ]).T


def my_fit(models, X_train, y_train):
    models['m_00'].fit(X_train, y_train)
    models['m_01'].fit(X_train, y_train)
    models['m_02'].fit(X_train, y_train)

    models['m_10'].fit(form_last_vec(models, X_train), y_train)
    return models['m_10']

In [28]:
rid = Ridge()
lin = LinearRegression()
lasso = Lasso()
print(rid, '\n\n', lin, '\n\n', lasso)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001) 

 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False) 

 Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)


### Ridge(Ridge+LinearRegression+Lasso)

In [64]:
models = {
    'm_00': Ridge(),
    'm_01': LinearRegression(n_jobs=8),
    'm_02': Lasso(),
    'm_10': Ridge(),
}

models['m_10'] = my_fit(models, X_train_part, y_train_part)
ridge_pred_up = models['m_10'].predict(form_last_vec(models, X_valid))

In [65]:
valid_mae = mean_absolute_error(y_valid, ridge_pred_up)
print(valid_mae)

0.7216629051721983


In [67]:
ridge_up = my_fit(models, X_train, y_train)

In [69]:
y_test = ridge_up.predict(form_last_vec(models, X_test))

### mean(Ridge+LinearRegression+Lasso)

In [60]:
random_state = 8123
models = {
    'm_00': Ridge(),
    'm_01': LinearRegression(n_jobs=8, ),
    'm_02': Lasso(),
    'm_10': Ridge(max_iter=10000, random_state=random_state),
}
models['m_10'] = my_fit(models, X_train_part, y_train_part)

mean_pred_up = form_last_vec(models, X_valid).mean(axis=1)
valid_mae = mean_absolute_error(y_valid, mean_pred_up)
print(valid_mae)

0.7337914681315665


In [474]:
models['m_10'] = my_fit(models, X_train_part, y_train_part)

mean_pred_up = form_last_vec(models, X_valid).mean(axis=1)
valid_mae = mean_absolute_error(y_valid, mean_pred_up)
print(valid_mae)

0.7338830055207693


In [61]:
models['m_10'] = my_fit(models, X_train, y_train)

### final mean(Ridge+LinearRegression+Lasso+tree)

In [94]:
def form_last_vec(models, X_vec):
    return np.array([
        models['m_00'].predict(X_vec[:,:1200]),
        models['m_01'].predict(X_vec[:,:1200]),
        models['m_02'].predict(X_vec[:,:1200]),
        models['m_03'].predict(X_vec),
    ]).T


def my_fit(models, X_train, y_train):
    models['m_00'].fit(X_train[:,:1200], y_train)
    models['m_01'].fit(X_train[:,:1200], y_train)
    models['m_02'].fit(X_train[:,:1200], y_train)
    models['m_03'].fit(X_train, y_train)
    
    return _

In [95]:
random_state = 8123
models = {
    'm_00': Ridge(),
    'm_01': LinearRegression(n_jobs=8, ),
    'm_02': Lasso(),
    'm_03': XGBRegressor(objective='rank:pairwise' , max_depth=15, min_child_weight=5, n_estimators=50, 
                         random_state=random_state, seed=random_state,),
    'm_10': Ridge(max_iter=10000, random_state=random_state),
}
_ = my_fit(models, X_train_part, y_train_part)

mean_pred_up = form_last_vec(models, X_valid).mean(axis=1)
valid_mae = mean_absolute_error(y_valid, mean_pred_up)
print(valid_mae)

0.7345850971716941


In [96]:
models['m_10'] = my_fit(models, X_train, y_train)

In [97]:
y_test = form_last_vec(models, X_test).mean(axis=1)

### overfitted tree(Ridge+LinearRegression+Lasso+tree)

In [99]:
def form_last_vec(models, X_vec):
    return np.array([
        models['m_00'].predict(X_vec[:,:1200]),
        models['m_01'].predict(X_vec[:,:1200]),
        models['m_02'].predict(X_vec[:,:1200]),
        models['m_03'].predict(X_vec),
    ]).T


def my_fit(models, X_train, y_train):
    models['m_00'].fit(X_train[:,:1200], y_train)
    models['m_01'].fit(X_train[:,:1200], y_train)
    models['m_02'].fit(X_train[:,:1200], y_train)
    models['m_03'].fit(X_train, y_train)
    
    return models['m_10'].fit(form_last_vec(models, X_train), y_train)

In [100]:
random_state = 8123
models = {
    'm_00': Ridge(),
    'm_01': LinearRegression(n_jobs=8, ),
    'm_02': Lasso(),
    'm_03': XGBRegressor(objective='rank:pairwise' , max_depth=15, min_child_weight=5, n_estimators=50, 
                         random_state=random_state, seed=random_state,),
    'm_10': XGBRegressor(objective='rank:pairwise' , max_depth=3, min_child_weight=5, n_estimators=5, 
                         random_state=random_state, seed=random_state,),
}

models['m_10'] = my_fit(models, X_train_part, y_train_part)

tree_pred_up = models['m_10'].predict(form_last_vec(models, X_valid))
valid_mae = mean_absolute_error(y_valid, tree_pred_up)
print(valid_mae)

0.8325799027853495


In [101]:
models['m_10'] = my_fit(models, X_train, y_train)

In [102]:
y_test = models['m_10'].predict(form_last_vec(models, X_test))

# submit

In [103]:
sub = pd.DataFrame()
sub['context_id'] = test['context_id']
sub['reply_id'] = test['reply_id']
sub['rank'] = - y_test
submission = sub.sort_values(by=['context_id', 'rank'])
del submission['rank']
submission.to_csv('./data/yandex-ml-stack-tree-tree-69.tsv',header=None, index=False, sep=' ')